In [1]:
#데이터셋 가져오기
#order 컬럼명 바꾸기 - 숫자로 되어 있는 부분에 Order (purchase event) 입력
import pandas as pd
#카테고리 전처리
import re
# 장바구니 분석
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import os
from collections import Counter

In [2]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총' #input("작업 파일 경로를 입력하세요")
os.chdir(input_path)
path=os.getcwd()
print(path)

#폴더 생성
x = dt.datetime.now()

file_name=f'[월간보고서] 재시도 {x.month}월 분석 결과'

if not os.path.exists(file_name):
    os.mkdir(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총


In [3]:
df= pd.read_clipboard(sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33915 entries, Model Code (v41) to SM-F721NZDEKOO,SM-F936NZKEKOO
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Jan 1, 2022 - Jan 31, 2022  33915 non-null  int64
 1   Feb 1, 2022 - Feb 28, 2022  33915 non-null  int64
 2   Mar 1, 2022 - Mar 31, 2022  33915 non-null  int64
 3   Apr 1, 2022 - Apr 30, 2022  33915 non-null  int64
 4   May 1, 2022 - May 31, 2022  33915 non-null  int64
 5   Jun 1, 2022 - Jun 30, 2022  33915 non-null  int64
 6   Jul 1, 2022 - Jul 31, 2022  33915 non-null  int64
 7   Aug 1, 2022 - Aug 31, 2022  33915 non-null  int64
 8   Sep 1, 2022 - Sep 30, 2022  33915 non-null  int64
 9   Oct 1, 2022 - Oct 31, 2022  33915 non-null  int64
 10  Nov 1, 2022 - Nov 30, 2022  33915 non-null  int64
 11  Dec 1, 2022 - Dec 31, 2022  33915 non-null  int64
dtypes: int64(12)
memory usage: 3.4+ MB


In [4]:
df=df[1:]
df=df.reset_index().rename(columns={"index": "Model Code (v41)"})
#df['2020.01~2022.11']=df.iloc[:,1:4].sum(axis=1)
df

,Model Code (v41),"Jan 1, 2022 - Jan 31, 2022","Feb 1, 2022 - Feb 28, 2022","Mar 1, 2022 - Mar 31, 2022","Apr 1, 2022 - Apr 30, 2022","May 1, 2022 - May 31, 2022","Jun 1, 2022 - Jun 30, 2022","Jul 1, 2022 - Jul 31, 2022","Aug 1, 2022 - Aug 31, 2022","Sep 1, 2022 - Sep 30, 2022","Oct 1, 2022 - Oct 31, 2022","Nov 1, 2022 - Nov 30, 2022","Dec 1, 2022 - Dec 31, 2022"
0,AX53A9313GEDT1,846,0,0,0,0,0,0,0,0,0,0,0
1,CFX-D100D,497,473,167,311,326,278,292,282,226,254,230,291
2,CFX-G100D,461,123,170,382,311,148,333,314,287,0,297,308
3,SM-R870NZSDKOO,444,79,100,117,73,41,49,15,2,0,1,2
4,DW50T4065SSS,403,324,307,230,180,140,154,184,184,64,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33909,"HGFG0020,HGFG0020,HGFG0023,HGFG0029",0,0,0,0,0,0,0,0,0,0,0,1
33910,"AF19B7932RFRZ,KQ65QB65-W15,RF85B9121APW",0,0,0,0,0,0,0,0,0,0,0,1
33911,"SF222498,SF179394,SF219812",0,0,0,0,0,0,0,0,0,0,0,1
33912,"RA-K15DUU39,RA-K15DUU39",0,0,0,0,0,0,0,0,0,0,0,1


In [5]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex_book=rex
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Category1D        37 non-null     object
 1   Category2D        37 non-null     object
 2   Category2D_Value  37 non-null     object
dtypes: object(3)
memory usage: 1016.0+ bytes


In [6]:

new_df=[]

for mobile in df.columns[1:]:
    print(f'************< {mobile} >******************')
    
    all=df[mobile].sum()
    one=df[~df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()
    two=df[df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()
    percent_one=round((df[~df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()/all)*100,2)
    percent_two=round((df[df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()/all)*100,2)

    
    
    print('전체: ',all)
    print('1개: ',one)
    print('2개이상: ',two)
    print("제품 1개 구매 비중: ",percent_one)
    print("제품 2개이상 구매 비중: ",percent_two)

    new_df1=[]
    for i in [all, one, two, percent_one,percent_two]:
        new_df1.append(i)
    new_df.append(new_df1)

# 데이터 저장3: 데이터 기준
pd.DataFrame(new_df,columns=['전체','1개','2개이상','제품 1개 구매 비중', '제품 2개이상 구매 비중'],
            index=[i[:3] for i in df.columns[1:]]).to_csv(f'{path}/{file_name}/[월간보고]기초통계량결과.csv', encoding='utf-8-sig')
pd.DataFrame(new_df,columns=['전체','1개','2개이상','제품 1개 구매 비중', '제품 2개이상 구매 비중'])

************< Jan 1, 2022 - Jan 31, 2022 >******************
전체:  31751
1개:  28470
2개이상:  3281
제품 1개 구매 비중:  89.67
제품 2개이상 구매 비중:  10.33
************< Feb 1, 2022 - Feb 28, 2022 >******************
전체:  114787
1개:  109046
2개이상:  5741
제품 1개 구매 비중:  95.0
제품 2개이상 구매 비중:  5.0
************< Mar 1, 2022 - Mar 31, 2022 >******************
전체:  63195
1개:  57845
2개이상:  5350
제품 1개 구매 비중:  91.53
제품 2개이상 구매 비중:  8.47
************< Apr 1, 2022 - Apr 30, 2022 >******************
전체:  44954
1개:  40080
2개이상:  4874
제품 1개 구매 비중:  89.16
제품 2개이상 구매 비중:  10.84
************< May 1, 2022 - May 31, 2022 >******************
전체:  64250
1개:  58925
2개이상:  5325
제품 1개 구매 비중:  91.71
제품 2개이상 구매 비중:  8.29
************< Jun 1, 2022 - Jun 30, 2022 >******************
전체:  43734
1개:  39133
2개이상:  4601
제품 1개 구매 비중:  89.48
제품 2개이상 구매 비중:  10.52
************< Jul 1, 2022 - Jul 31, 2022 >******************
전체:  42746
1개:  37925
2개이상:  4821
제품 1개 구매 비중:  88.72
제품 2개이상 구매 비중:  11.28
************< Aug 1, 2022 - Aug 31, 2022 >**

,전체,1개,2개이상,제품 1개 구매 비중,제품 2개이상 구매 비중
0,31751,28470,3281,89.67,10.33
1,114787,109046,5741,95.00,5.00
2,63195,57845,5350,91.53,8.47
3,44954,40080,4874,89.16,10.84
4,64250,58925,5325,91.71,8.29
5,43734,39133,4601,89.48,10.52
6,42746,37925,4821,88.72,11.28
7,192328,183012,9316,95.16,4.84
8,132916,126904,6012,95.48,4.52
9,72186,67276,4910,93.20,6.80


In [8]:
for rex_list in rex.Category1D.unique():
    print(f'<<<{rex_list}>>')
    new_rex=rex[rex['Category1D']==rex_list]
    new_rex=new_rex.reset_index()
    final_year={}

    for mobile in df.columns[1:]:
        print(f'************< {mobile} >******************')

        mobile_df=df[df[mobile]!= 0]
        #print('order 0제외 행개수:',mobile_df.shape[0])
        mobile_df_1=mobile_df[~mobile_df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')]
        mobile_df_2=mobile_df[mobile_df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')]
        
    

        final_order={}
        code_book={'전체':mobile_df,'1개':mobile_df_1,'2개이상':mobile_df_2}
        
        for key, value in code_book.items():
            value=value.reset_index(drop=True)
            data=value
            a = data['Model Code (v41)'].tolist()
            b = ','.join(a) #리스트 문자열로
            c = b.split(',') #문자열 개별 문자로 리스트화 
            model = list(set(c)) #중복 제거

            #전체 SKU 담을 딕셔너리
            dic_category = {}

            for m in model :
                flag = 1
                for i in range(len(new_rex)):
                
                    r = re.compile(new_rex.Category2D_Value[i])
                    if r.search(m):
                        dic_category[m] = new_rex.Category1D[i] +'_'+ new_rex.Category2D[i]  #new_rex.Category1D[i] +'_'+  #sku 규칙을 알고자한다면, r.search(m).group()
                        flag = 0
                        break
                if flag :
                    dic_category[m] = '기타'
            #데이터셋 만들기
            dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

            for i in range(len(data)) :
                sku = data['Model Code (v41)'][i]
                revised = sku.split(",")
                code = []
                for j in range(len(revised)) :
                    code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
                
                for k in range(data[mobile][i]) :   #order 수만큼 담기
                    dataset.append(code) 
            print(f'<<<{key}>>>')
            print('기존 dataset 행 수: ',len(dataset))

            #기타제거
            ho = []
            rm_set = {'기타'}
            for s in dataset:
                arr_new = [i for i in s if i not in rm_set]
                ho.append(arr_new) 
            dataset=ho
            
            print('기타 제거 후, 행 수: ',len(dataset))

            select_mobile=[s for s in dataset if any(rex_list in l for l in s)]
            print(f'{rex_list}이 포함된 행수: ',len(select_mobile))  
        

            # 실제 구매 비중
            per_order={}
    
            for  PRODU in new_rex['Category2D']:
                df_count=[s for s in dataset for i in s if PRODU in i]
                per_order[PRODU]=len(df_count)
            final_order[key]=per_order
        final_year[mobile]=final_order

    year_order=pd.DataFrame.from_dict(final_year, orient="index").stack().to_frame()
 
# 데이터 저장2: 모바일 항목 내에 카운트   
    pd.DataFrame(year_order[0].values.tolist(), index=year_order.index).to_csv(f'{path}/{file_name}/[월간보고]{rex_list}_전체 주문수 결과.csv', encoding='utf-8-sig')


<<<모바일>>
************< Jan 1, 2022 - Jan 31, 2022 >******************
<<<전체>>>
기존 dataset 행 수:  31751
기타 제거 후, 행 수:  31751
모바일이 포함된 행수:  13871
<<<1개>>>
기존 dataset 행 수:  28470
기타 제거 후, 행 수:  28470
모바일이 포함된 행수:  12832
<<<2개이상>>>
기존 dataset 행 수:  3281
기타 제거 후, 행 수:  3281
모바일이 포함된 행수:  1039
************< Feb 1, 2022 - Feb 28, 2022 >******************
<<<전체>>>
기존 dataset 행 수:  114787
기타 제거 후, 행 수:  114787
모바일이 포함된 행수:  98090
<<<1개>>>
기존 dataset 행 수:  109046
기타 제거 후, 행 수:  109046
모바일이 포함된 행수:  94609
<<<2개이상>>>
기존 dataset 행 수:  5741
기타 제거 후, 행 수:  5741
모바일이 포함된 행수:  3481
************< Mar 1, 2022 - Mar 31, 2022 >******************
<<<전체>>>
기존 dataset 행 수:  63195
기타 제거 후, 행 수:  63195
모바일이 포함된 행수:  44538
<<<1개>>>
기존 dataset 행 수:  57845
기타 제거 후, 행 수:  57845
모바일이 포함된 행수:  41434
<<<2개이상>>>
기존 dataset 행 수:  5350
기타 제거 후, 행 수:  5350
모바일이 포함된 행수:  3104
************< Apr 1, 2022 - Apr 30, 2022 >******************
<<<전체>>>
기존 dataset 행 수:  44954
기타 제거 후, 행 수:  44954
모바일이 포함된 행수:  23239
<<<1개>>>
기존 data